In [ ]:
!pip install missingpy
!pip install sklearn
!pip install tensorflow
!pip install scikit-learn==v1.0.2

In [1]:
import pandas as pd
import numpy as np
from numpy.random import default_rng
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import scipy
import sys
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
import random
from IPython.display import display
from sklearn import preprocessing
from missingpy import MissForest

## Data Analysis

In [2]:
colums = ['age','workclass','fnlwgt','education','education-num','marital-status','occupation','relationship','race','sex','capital-gain','capital-loss','hours-per-week','native-country', '50K']
adult_train = pd.read_csv('data/adult_train.txt', sep=',', header=0, names=colums)
adult_test = pd.read_csv('data/adult_test.txt', sep=',', header=0, names=colums)

# 32561 instances, mix of continuous and discrete    (train=26048, test=6513)

In [3]:
adult_train

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,50K
0,46,Private,415051,Some-college,10,Married-civ-spouse,Sales,Husband,Black,Male,0,0,60,United-States,>50K
1,28,Private,76129,HS-grad,9,Never-married,Machine-op-inspct,Unmarried,White,Female,0,0,40,Guatemala,<=50K
2,19,?,192773,Some-college,10,Never-married,?,Own-child,White,Female,0,0,35,United-States,<=50K
3,43,Private,160369,HS-grad,9,Never-married,Machine-op-inspct,Own-child,White,Male,0,0,40,United-States,<=50K
4,22,?,121070,Some-college,10,Never-married,?,Own-child,White,Male,0,0,35,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26043,32,Private,176992,10th,6,Never-married,Craft-repair,Unmarried,White,Male,0,0,40,United-States,<=50K
26044,40,Private,55191,Some-college,10,Never-married,Adm-clerical,Not-in-family,Black,Female,0,0,40,United-States,<=50K
26045,79,Private,124744,Some-college,10,Married-civ-spouse,Prof-specialty,Other-relative,White,Male,0,0,20,United-States,<=50K
26046,50,Private,192203,11th,7,Married-civ-spouse,Transport-moving,Husband,White,Male,0,0,40,United-States,<=50K


In [4]:
adult_test

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,50K
0,54,Private,175262,7th-8th,4,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,<=50K
1,17,Private,295991,10th,6,Never-married,Adm-clerical,Own-child,White,Female,0,0,20,United-States,<=50K
2,60,Private,240521,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,White,Male,7298,0,40,United-States,>50K
3,26,Private,236068,Some-college,10,Never-married,Sales,Other-relative,White,Female,0,0,20,United-States,<=50K
4,35,Private,186934,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,7688,0,50,United-States,>50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6508,33,Self-emp-inc,374905,HS-grad,9,Married-civ-spouse,Sales,Husband,White,Male,0,0,50,United-States,>50K
6509,41,Private,167106,Bachelors,13,Married-civ-spouse,Adm-clerical,Husband,Asian-Pac-Islander,Male,3103,0,35,Philippines,>50K
6510,60,Private,223696,1st-4th,2,Divorced,Craft-repair,Not-in-family,Other,Male,0,0,38,Dominican-Republic,<=50K
6511,31,Private,101562,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Wife,White,Female,0,0,55,United-States,<=50K


In [5]:
# to see missing values in adult dataset

print(adult_train.isnull().sum()) # per label
print(adult_train.isnull().sum().sum()) # total
print(adult_test.isnull().sum())

# the missing values are not being recognized because they are written as "?"

age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
50K               0
dtype: int64
0
age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
50K               0
dtype: int64


In [6]:
# to know the type and form of missing values in adult dataset
print('missing values:'+adult_train["workclass"][2]+')') # in row 4, column 1, we have a missing value and we will check its time and form
type(adult_train["workclass"][2])

# we saw that the missing values here are written as strings "?"

missing values:?)


str

In [7]:
# Making a list of possible missing value types 
missing_values = ["?", "n/a", "na", "Nan", " ", "nan", "NaN"]

# to set missing values to "NaN" in order to be recognized
df_train = pd.read_csv("data/adult_train.txt", sep=',', header=0, names=colums, na_values = missing_values)
df_test = pd.read_csv("data/adult_test.txt", sep=',', header=0, names=colums, na_values = missing_values)

df_train

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,50K
0,46,Private,415051,Some-college,10,Married-civ-spouse,Sales,Husband,Black,Male,0,0,60,United-States,>50K
1,28,Private,76129,HS-grad,9,Never-married,Machine-op-inspct,Unmarried,White,Female,0,0,40,Guatemala,<=50K
2,19,NaN,192773,Some-college,10,Never-married,NaN,Own-child,White,Female,0,0,35,United-States,<=50K
3,43,Private,160369,HS-grad,9,Never-married,Machine-op-inspct,Own-child,White,Male,0,0,40,United-States,<=50K
4,22,NaN,121070,Some-college,10,Never-married,NaN,Own-child,White,Male,0,0,35,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26043,32,Private,176992,10th,6,Never-married,Craft-repair,Unmarried,White,Male,0,0,40,United-States,<=50K
26044,40,Private,55191,Some-college,10,Never-married,Adm-clerical,Not-in-family,Black,Female,0,0,40,United-States,<=50K
26045,79,Private,124744,Some-college,10,Married-civ-spouse,Prof-specialty,Other-relative,White,Male,0,0,20,United-States,<=50K
26046,50,Private,192203,11th,7,Married-civ-spouse,Transport-moving,Husband,White,Male,0,0,40,United-States,<=50K


In [8]:
# check if the missing values are now recognized
print(df_test.isna().sum())  # yes :)

print('missing values in train: '+str(df_train.isna().sum().sum()))
print('missing values in test: '+str(df_test.isna().sum().sum()))

age                 0
workclass         365
fnlwgt              0
education           0
education-num       0
marital-status      0
occupation        366
relationship        0
race                0
sex                 0
capital-gain        0
capital-loss        0
hours-per-week      0
native-country    107
50K                 0
dtype: int64
missing values in train: 3424
missing values in test: 838


In [9]:
# we will check the missing values row by row, column by column and save its index
index_train = []
for row in adult_train.iterrows():
    for j in row[1]:
        if j == '?':
            index_train.append(row[0])

index_test = []
for row in adult_test.iterrows():
    for j in row[1]:
        if j == '?':
            index_test.append(row[0])

#print(index) -> we have duplicated instances because there are instances with more than one missing value
# to clean duplicated index
seen = set()
index_train = [x for x in index_train if x not in seen and not seen.add(x)]  
seen = set()
index_test = [x for x in index_test if x not in seen and not seen.add(x)]  

print('number of index with missing values in train: '+str(len(index_train)))
print('number of index with missing values in test: '+str(len(index_test)))
print(str((len(index_test)+len(index_train))/(len(adult_train)+len(adult_test))*100)+'% have missing values.')
# 30162 if instances with unknown values are removed (train=24116, test=6046)

number of index with missing values in train: 1932
number of index with missing values in test: 467
7.367709836921471% have missing values.


We saw that 7% of instances have missing values and then we will remove them. But first, we will check the class distribution to be possible compare it before and after that elimination.

In [10]:
# Class distribution

g50_train=[]
l50_train=[]
for i in range(0, len(df_train)):
    if df_train['50K'][i]=='<=50K':
        l50_train.append(i)
    if df_train['50K'][i]=='>50K':  
        g50_train.append(i)

g50_test=[]
l50_test=[]
for i in range(0, len(df_test)):
    if df_test['50K'][i]=='<=50K.':
        l50_test.append(i)
    if df_test['50K'][i]=='>50K.':  
        g50_test.append(i)

        
# Probability for the label '>50K':
print('Probability for the label >50K: '+str((len(g50_train)+len(g50_test))/(len(df_train)+len(df_test))*100) + '%')
# Probability for the label '<=50K':
print('Probability for the label =<50K: '+str((len(l50_train)+len(l50_test))/(len(df_train)+len(df_test))*100) + '%')


Probability for the label >50K: 19.308375049906328%
Probability for the label =<50K: 60.68916802309512%


Now, we are able to remove the instances which have missing values.

In [11]:
# Remove instances with unknown values

df_train.drop(index_train, inplace=True)
df_test.drop(index_test, inplace=True)

# instances with unknown values were removed and now we have 30162 (train=24116, test=6046)

In [12]:
# to then compare the dataset D with the dataset with imputed values, we will make a copy of the df and order its index

D_train = df_train.copy()
D_train = D_train.sort_index(ignore_index = True)

D_test = df_test.copy()
D_test = D_test.sort_index(ignore_index = True)
display(D_test)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,50K
0,54,Private,175262,7th-8th,4,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,<=50K
1,17,Private,295991,10th,6,Never-married,Adm-clerical,Own-child,White,Female,0,0,20,United-States,<=50K
2,60,Private,240521,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,White,Male,7298,0,40,United-States,>50K
3,26,Private,236068,Some-college,10,Never-married,Sales,Other-relative,White,Female,0,0,20,United-States,<=50K
4,35,Private,186934,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,7688,0,50,United-States,>50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6041,33,Self-emp-inc,374905,HS-grad,9,Married-civ-spouse,Sales,Husband,White,Male,0,0,50,United-States,>50K
6042,41,Private,167106,Bachelors,13,Married-civ-spouse,Adm-clerical,Husband,Asian-Pac-Islander,Male,3103,0,35,Philippines,>50K
6043,60,Private,223696,1st-4th,2,Divorced,Craft-repair,Not-in-family,Other,Male,0,0,38,Dominican-Republic,<=50K
6044,31,Private,101562,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Wife,White,Female,0,0,55,United-States,<=50K


In [13]:
D_train.to_csv("../Fair-Explainable-Analytics-Seminar/data/D_train.csv")
D_test.to_csv("../Fair-Explainable-Analytics-Seminar/data/D_test.csv")

In [14]:
# Class distribution without unknowns

g50_train=[]
l50_train=[]
for i in df_train.index:
    if df_train['50K'][i]=='<=50K':
        l50_train.append(i)
    if df_train['50K'][i]=='>50K':  
        g50_train.append(i)

g50_test=[]
l50_test=[]
for i in df_test.index:
    if df_test['50K'][i]=='<=50K.':
        l50_test.append(i)
    if df_test['50K'][i]=='>50K.':  
        g50_test.append(i)

        
# Probability for the label '>50K':
print('Probability for the label >50K without unknowns: '+str((len(g50_train)+len(g50_test))/(len(df_train)+len(df_test))*100) + '%')
# Probability for the label '<=50K':
print('Probability for the label <=50K without unknowns: '+str((len(l50_train)+len(l50_test))/(len(df_train)+len(df_test))*100) + '%')

Probability for the label >50K without unknowns: 19.962204097871496%
Probability for the label <=50K without unknowns: 59.9927060539752%
